# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [21]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [22]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [23]:
browser = webdriver.Chrome(executable_path = './chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # 捲動畫面
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待畫面捲動時間
    time.sleep(SCROLL_PAUSE_TIME)

    # 取得新的畫面高度
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    # 畫面捲動後沒有更新高度就跳出回圈
    if new_height == last_height:
        break
    
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [24]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [25]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 悠遊卡電子支付上線「嗶乘車」　董座陳亭如：持續努力讓蘋果加入

▲悠遊付記者會。（圖／記者楊亞璇攝）
記者楊亞璇／台北報導
悠遊卡公司23日宣布，電子支付「悠遊付」3月23日起全面開放註冊，未來在悠遊卡、悠遊付「一卡一付」的整合下，最為重要的功能為「嗶乘車」，只要民眾持有NFC功能之Android6.0以上手機就能感應進出雙北捷運、雙北公車、YouBike，不過不支援Apple。
悠遊卡公司表示，只要下載「Easy Wallet」App，完成實名制註冊、綁定金融帳戶後就能開始使用，5大功能包括，第一，嗶乘車，第二，帳戶間轉帳，第三，綁定悠遊卡，可透過「悠遊付」自動加值悠遊卡，並管理卡片資訊。第四為QR Code掃碼消費支付、收款及線上付款，最後一個功能為以「悠遊付」申辦虛擬台北卡會員身分、繳納台北市自來水費、台北市立聯合醫院醫療費及台北市停車費。
在小額支付方面，悠遊付首波將有頂好超市、萊爾富、春芳號等連鎖系統上線，夜市商圈則有艋舺夜市、寧夏夜市可以用手機掃碼QR code完成支付，即將上線的台灣大車隊同樣也有悠遊付提供服務。
陳亭如強調，悠遊卡不會沒有優惠，但是補貼不能帶來忠誠度，會結合市政府，把人家的忠誠單位結合，悠遊卡會考量疫情發展，也不會貿然鼓勵大家乘車。
首波支援悠遊付金融帳戶儲值的銀行業者，包括第一銀行、彰化銀行、台北富邦銀行、國泰世華銀行、新光銀行、永豐銀行、台新銀行及中國信託銀行。自3月23日起至4月30日活動期間，綁定銀行帳戶且自動儲值至悠遊付帳戶，即可獲得儲值金回饋，每人限回饋1次，限量回饋。
----------------------------------------------------------------------

[1] 基隆、汐止大停電「紅綠燈全停擺」　台電：緊急搶修已全部復電

▲基隆、汐止等地今（23）日上午大停電，台電派員搶修後已全部復電。（圖／台電提供）
記者林淑慧／台北報導
基隆市、瑞芳與新北市汐止區居民看過來！台電公司今（23）日表示，因八堵變電所發生事故，上述地區今天早上10點10分後大停電，路上的紅綠燈號誌都停擺，經台電派員緊急搶救，所有停電戶已於10時57

▲ 新台幣兌美元今天開盤貶值1.48角。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（23）日開盤重貶1.48角、以30.45元兌1美元開出。
新台幣匯率上周五（20日）收在30.302元兌1美元，今天開盤以30.45元兌1美元開出、貶值1.48角。
截至上午9點04分，新台幣來到30.429元兌1美元，貶值1.27角。
受到市場持續恐慌影響，台股今日早盤也大幅受挫，開盤以下跌208點開出，隨後持續下挫，跌點擴大到360點以上，並跌破8900點整數關卡。
----------------------------------------------------------------------

[15] 疫情蔓延、紓困措施未通過　美股三大指數期貨皆出現跌停　

▲美股。（圖／路透社）
記者余弦妙／台北報導
美國新冠肺炎疫情蔓延，美國國會又未能就支出法案達成協議，美國三大指數期貨在台灣時間周一早上出現暴跌，甚至一度跌停。
道瓊期指截至台灣時間9點，為18109.5點，下跌930.5點或4.89%。
另外，那斯達克期指下跌310.75點或4.46%，為6658.25點。
標普500期指則跌109.38點或4.78%%，來到2179.12點。
新冠肺炎疫情持續在美國延燒，總統川普提出的經濟紓困法案卻遭民主黨參議員封殺，美股期指周一再度出現崩跌行情，跌幅並達5%，再度觸發跌停機制。
美國共和黨所提規模高達2兆美元的經濟紓困，遭民主黨參議員封殺，因民主黨認為，這項提案無法充分協助數以百萬計受疫情衝擊的美國民眾，受到提案無法過關的衝擊，美股期指今日開盤應聲重挫。
----------------------------------------------------------------------

[16] 台股失守8900點！早盤重挫361點　國安基金面臨考驗

▲ 台股。（資料照／記者湯興漢攝）
記者李瑞瑾／台北報導
新冠肺炎（COVID-19）疫情在歐美肆虐，市場人心惶惶，美股持續走低，美股期指今早一度熔斷，台股上周五雖反彈收高，但仍不敵國際股市同步下挫壓力，今（23）日開盤下跌208點、以9025點開出。
截至上午9點01分，指數來到8872點、下跌361點。
（更新）截至上午9點12分，指數一度下跌483點、最低來到8750點。


▲統一發票兌獎。（圖／記者許力方攝）
記者吳靜君／台北報導
統一發票1、2月期即將在週三（25日）開獎，民眾又可以享受兌獎的樂趣！但是別忘了喔！108年11、12月期統一發票1,000萬元特別號中獎「59647042」中獎張數共計有13張，其中有5張尚未兌領，其中花35元在7-11(統一超商）學屋門市買零食，就中千萬大獎的幸運兒，也尚未兌領。
財政部表示，去年11、12月期的中獎統一發票兌獎時間是只到5月5日，所以還未兌獎的民眾，記得要去兌領。
根據財政部所公布的資料顯示，千萬特別獎還未兌領有5張發票，台北市有1張，為台北市大安區的遠傳電信開出，消費者儲值300元點數儲值費。
新北市也有1張，則是在新北市土城區的台塑延吉加油站，加79元的汽油。桃園市則有1張，在桃園市中壢區的7-11學屋門市購買35元手摘果物就中了大獎。
台南市的1張同樣在7-11消費，在台中市南屯區黎明東門市購買照燒雞糖心蛋三明治與熱拿鐵，花費119元。
高雄市1張，則是在高雄鳥松區高雄長庚醫院的摩斯漢堡買熱抹茶拿鐵80元，就中的千萬統一發票大獎。
▲108年11、12月統一發票未兌領清冊。（表／記者吳靜君、資料來源／財政部）
另外，108年11、12月期200萬元特獎中獎號碼為「01260528」，中獎張數共計18張，有10張尚未兌領。
根據財政部的未領清冊顯示，台北市有2張，其中一張是台北市大安區的優勢領航科技開出，消費者花400元儲值「Ｇ妹遊戲」；另一張則是App Store首次開出的大獎，消費者在iTunes購買數位影音150元。基隆市也有1張，則是在基隆市七堵區的OK便利商店永富店裡，購買樂事九州岩燒海苔，花費30元。
新北市也有2張，一張則是在新北市淡水區7-11金沙門市購買菸品以及打火機，總花費為120元；另一張則是新北市中和區的在美聯社中和民利店購買台啤，花費共計154元。
宜蘭縣則也有1張，則是在宜蘭縣宜蘭市的中油廣進加油站洗車，花費70元；以及新竹縣也有1張，則是在新竹縣新埔鎮的7-11新普門市購買報紙和食品等，花費160元。台南市有1張，為在台南市新營區的全家新營車站店購買大口法式奶油起司雞排飯糰以及巧克力牛奶，花費59元。
高雄市則有2張，分別是在高雄鳥松區的成旺金香鋪購買金紙1680元；以及高雄三民區的生力美食購買麵包花了252元。財政部表示，去年的11、12月中獎的統一

▲經濟部工業局攜手工研院、榮剛、久陽等大廠，打造智能金屬產業育才平台，透過產學合作培養人才。(圖／經濟部提供)
記者林淑慧／台北報導
年後轉職潮一波接一波，為加速金屬產業邁向智慧製造，經濟部工業局攜手工研院、榮剛、久陽等大廠，打造智能金屬產業育才平台，促成企業透過產學合作模式，打造金屬產業新秀MVP，協助金屬產業拼轉型。
為達成產品高值化、製造智慧化及管理數位化，智能金屬產業育才平台協助媒合包括鋼鐵(榮剛、天聲)、扣件(久陽)、鑄造(精工、勤美)及模具(龍畿)等產業廠商，鏈結國立高雄大學、國立高雄科技大學及明志科技大學，共同培育金屬產業優質人才。
官員指出，此6家企業結合大學校院開立客製化專業課程，依企業人才需求量身訂製培育企業所需人才，同時活絡產業人才孕育橋梁，促成企業透過產學合作模式，並以企業高值化及智能化技術需求為導向，規劃專題研究。
舉例來說，榮剛材料開設「智慧聯網系統於鋼鐵廠的應用」及勤美公司「MES資訊數據分析」等智慧化相關課程；另在專題研究方面，廠商依需求提出，如天聲工業「產線數據自動收集系統」及「自動化生產履歷系統」與久陽精密「螺絲線材的智能化成形」等專題，由企業提供師資及專題指導，結合學校資源使實務與理論能夠相互結合，達到深耕育才，量穩質精之效益。
經濟部工業局表示，智能金屬產業育才平台，可望輔助企業育才時，達成縮短學用落差及解決人才缺口之效益，企業可透過政府資源強化產業與周邊大學校院的鏈結，網羅優質人才及讓學生及早認識產業，以期達成「企業預約人才，學生銜接就業，學校發展特色」的三贏局面。
----------------------------------------------------------------------

[35] 元大S＆P原油正2遭質疑　金管會：投資人自認權益受損可申訴

▲市場傳出元大證券「自肥」一事，金管會今天澄清，發布函令前並未向業者透露。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
國際油價慘跌，元大S&P原油正2（00672L）淨值低於2元，已達下市門檻，金管會19日發布函令放寬標準，暫時解除下市危機，不過市場傳出元大在發布函令當天買該檔ETF逾12萬張，引起外界質疑有「自肥」的疑慮，金管會今（21）日表示，發布前並未向業者透露，且未與元大證券有任何接觸，絕不會容許證券商藉機獲利。
市場質疑金管會的特赦令走漏風

▲綜藝天王豬哥亮入殮擁恆文創園區，園區創辦人陳振豐，再獲無罪。（圖／《ETtoday新聞雲》資料照）
記者吳銘峯／台北報導
國統集團總裁陳振豐被控涉入2000年爆發的基隆市「大水窟弊案」，一審無罪，但二審遭判刑8月。案經撤銷發回後，高等法院更一審駁回檢方上訴，改判決陳振豐無罪，依照《刑事妥速審判法》規定，檢方上訴受限，陳振豐幾乎無罪確定。對此，陳振豐發出聲明，強調本案纏訟近20年遲來的正義，終於沉冤得雪。
陳振豐身兼薇閣汽車旅館老闆、國統集團總裁、擁恆文創園區創辦人，其家族企業為股票上市公司高意建設開發公司，高意公司更是另一家上市公司基泰建設的大股東。但檢方指控，兩家公司在2000年左右開發基隆「大水窟」地區，陳振豐當時為了開發擁恆文創園區，涉嫌將9000萬元回扣，酬謝給基泰建設總經理馮先勉，馮先勉還因此遭檢方收押禁見，陳振豐最後被依照「背信罪」提起公訴。
陳振豐一審獲判無罪，但二審改認定有罪，判刑8月。上訴最高法院後，最高法院撤銷發回，全案由高等法院更一審，更一審最後以查無實證為由，駁回檢方上訴，維持陳振豐的一審無罪判決。依照依照《刑事妥速審判法》規定，檢方上訴第三審受限，陳振豐幾乎無罪確定。
對此，國統集團發布聲明，強調本案纏訟近20年，終於確保陳振豐清白，遲來的正義再次證明，國統公司開發擁恆文創園區一切程序合法、合理、合情。集團發言人王志欽表示，沉冤昭雪，肯定了陳振豐及國統公司致力於擁恆文創園區的經營管理，恪遵法規，嚴守分際。未來集團將更以「家」為核心理念，打造「來者以遊，生者以慰，逝者以安」的人間淨土。
----------------------------------------------------------------------

[46] 快訊／美股開盤道瓊「紅轉綠」　再度跌破2萬點

▲美股再度跌破2萬點。（圖／達志影像／美聯社）
記者鄒鎮宇／綜合報導
美股三大指周五開盤，道瓊工業指數開盤一度上漲逾170.23點或0.85％，不久下跌114.76點，約0.57%，跌至19933.10點；標普500指和那斯達克綜合指數漲幅一度都超過2％，但標普轉綠下跌，報2381.75點。
美股開盤由科技股帶動，蘋果漲1.4%、微軟漲2.26%、亞馬遜漲3.1%、谷歌漲0.8%、臉書漲3.5%，道指一度漲超300點，但隨即紅轉綠下跌。
►酒精乾洗手下殺3折！

蘋果重拾超寬頻技術，無疑是ＵＷＢ捲土重來的市場代表，也掀起新應用的無限想像，帶動國內如瑞昱、聯發科、智原等ＩＣ設計廠成為焦點。
文／吳旻蓁
蘋果每年的iPhone發表會，絕對可以稱之為手機界的頭等大事。而去年的iPhone 11系列機款，吸睛的可不只有被稱為「三眼怪」的三眼鏡頭而已，另一個引起業界廣泛討論的，雖在發表會上並沒有特別被提及，不過似乎可從蘋果官網iPhone 11的一段介紹文字中嗅到商機。
室內定位需求與日俱增
「Apple全新設計的Ｕ１晶片將超寬頻技術用於空間感知」，就是這顆神秘的Ｕ１晶片掀起市場熱切關注。據了解，Ｕ１晶片讓iPhone可以支援超寬頻技術（Ultra Wideband, UWB），增強空間感知與強化感知能力，使周圍同樣有搭載Ｕ１晶片的裝置能更精準地定位，未來將可帶來更豐富的室內定位及室內導航功能，「就像是定位範圍達客廳大小的ＧＰＳ一樣」蘋果這麼形容。
其實超寬頻並非是近幾年才發展的新技術，ＵＷＢ出現在商用市場已快二○年的時間。ＵＷＢ是一種短距離無限通訊技術，與傳統通信技術有極大差異，它不需使用傳統通信體制中的載波，而是通過發送和接收具有奈秒或奈秒級以下的極窄脈衝來傳輸數據，傳輸速率可高達一一○至四八○Mbps，且其最大特色是能精準定位，傳輸距離在十公尺內，探測的精度最小能縮到五公釐以下，遠勝過藍牙和WiFi的一公尺精確度。
然而過去因ＵＷＢ的成本較藍牙和WiFi高，加上生態系尚不完整，因此還未發展起來就沉寂了。以往，蘋果透過旗下產品帶動技術與應用風潮並非新鮮事，像是觸控技術、ＴＷＳ等也都不是新技術，卻因著蘋果冷飯熱炒而大放光彩，也讓市場對於Ｕ１晶片將開創的新應用產生許多期待。
而這個期待似乎並不遙遠，根據天風國際證券知名分析師郭明錤最新的報告中透露，繼iPhone之後，蘋果有計畫於今年的秋季發表會上推出一款搭載ＵＷＢ的藍牙定位追蹤裝置，可能稱為AirTags或Apple Tag，並預期在今年開出千萬台的量產，其主要功能是用來定位和追蹤物品位置，據悉，只要將AirTags黏在物品上，就算不小心忘記放在哪，也只要透過iPhone找到，甚至還能整合ＡＲ功能，讓用戶直接透過螢幕，看到物品的具體位置。
ＵＷＢ再創新藍海
此次蘋果之所以重拾ＵＷＢ，看中的就是正在快速發展的物聯網時代，因為當聯網裝置越來越多、越來越繁雜，用戶對裝置間的精準溝通、速度等

▲義大利目前正在考慮將快破產的義大利航空收歸國有。（圖／路透）
記者康心禹／綜合報導
新冠肺炎（COVlD-19）疫情肆虐，全球航空公司大受打擊，美國財政部長梅努欽（Steven Mnuchin）表示，新冠肺炎對航空業造成的影響，比2011年911之後美國全面關閉領空還嚴峻。國際航空運輸協會（IATA）指出，全球航空業者需要2000億美元（約新台幣6.1兆元）的政府支援求生。
綜合外媒報導，據了解，義大利目前正在考慮將快破產的義大利航空收歸國有，瑞士和丹麥則端出2.75億歐元擔保金來支撐北歐航空公司（SAS）。
國際航空運輸協會執行長迪朱尼艾克（Alexandre de Juniac）表示，遏制疫情擴散是各國政府當務之急，就全球範圍來看，估計業者需要高達2000億美元緊急支援。
▲美國財政部長梅努欽（Steven Mnuchin）。（圖／路透）
據《華爾街日報》報導，美國的航空公司正在尋求政府提供超過500億美元的財政援助，這是9月11日襲擊事件後航空業紓困規模的3倍。
國際航空運輸協會的首席經濟學家皮爾斯（Brian Pearce）稱，許多航空公司的現金正在耗盡，75%的航空公司的現金只夠維持不到3個月的必要固定成本開支。航空業智庫「亞太航空中心」（CAPA）也示警，全球大部分航空業者恐在5月底宣布破產。
----------------------------------------------------------------------

[69] 印尼禁止勞工出國工作　勞動部有3大建議供雇主參考

▲勞動部。（圖／記者余弦妙攝）
記者余弦妙／台北報導
新冠肺炎疫情持續延燒，印尼勞動部指出20日起將暫時停止輸出所有移工，目的在於保護國內外移工。對此，我國勞動部則鼓勵雇主採續聘及國內承接維持移工人數，這樣也能防止人員往來出現防疫破口。
勞動部跨國勞動力管理組組長薛鑑忠表示，目前在台印尼移工有27.6萬人，占了總外籍移工的38％，27.6萬人中看護工就有20萬，佔了約70％；外籍漁工1.2萬人，其中印尼籍約9000多人，同樣佔了約70%；另外43.9萬外籍製造業移工中，印尼籍6.5萬人約佔了15％。
雖然目前印尼禁止移工輸出，不過薛鑑忠也說，目前台灣還有泰國、越南、菲律賓等國家的移工可以輸入，因此目前除了鼓勵雇主繼續續聘或國內承接原本印尼移工外，雇主也能選擇聘

▲ 逐筆交易下周一正式上路。（圖／記者李瑞瑾攝）
記者李瑞瑾／台北報導
台股盤中全面逐筆交易於下周一（23日）正式上路！新制委託有何注意事項，證交所也一次整理給投資大眾解惑。
開盤（08:30~09:00）—集合競價
僅接受限價當日有效（ROD）委託，使用市價單、IOC及FOK委託將會被退單。
盤中（09:00~13:25）—逐筆交易
有六種委託方式（除限價ROD委託外，新增限價IOC、限價FOK、市價ROD、市價IOC及市價FOK）。
▲ 逐筆交易6種委託。（圖／證交所）
市價單優先限價單（含漲停買進、跌停賣出）成交，即盤中市價買單優先漲停買進委託、市價賣單則優先跌停賣出委託，投資大眾要留意市價單與限價單之優先順序。
市價單、IOC及FOK委託，可能成交價如觸及瞬間價格穩定措施上、下限，需留意成交狀況及是否有被退單之情形。
另外，雖盤中原則上皆採集合競價，但如遇瞬間價格穩定措施，則會暫緩2分鐘後以「集合競價」撮合成交。
收盤（13:25~13:30）—集合競價
僅接受限價當日有效（ROD）委託，使用市價單、IOC及FOK委託將會被退單。
委託改價功能
為了提升投資人委託效率，證交所這次新增了限價委託的改價功能，全時段均可直接對限價委託改價，但限價不可改為市價、市價也不可改為限價。
此外，平盤下不得放空之有價證券，選擇以「借券賣出」或「融券賣出」之投資人，不得將價格改為平盤以下。
----------------------------------------------------------------------

[82] 宏達電去年第4季虧損23.7億元　每股稅後淨損2.87元

▲宏達電新店總部。（圖／記者湯興漢攝）
記者姚惠茹／台北報導
宏達電（2498）今（20）日公布2019年第4季營收18.1億元，毛利率25.7%，較上季的22.9%，增加2.8個百分點，創2017年第4季以來，連續8個季度毛利率成長，稅後淨損23.7億元，每股稅後虧損2.87元。
回顧去年第4季營運，宏達電表示，VIVE虛擬實境方面，VIVE Cosmos年底在全球開賣，作為最新款高規電腦驅動PC VR產品，使用者不再需要安裝定位基地台，就能享受便利且絕佳的虛擬實境體驗，同時可換式面板模組設計可隨著使用者的需求，調整或擴充頭戴式顯示器的功能。
藝術領域應用方面，宏達電分享，攜手

▲勞動部勞動利發展署設置補貼懶人專區。（圖／勞動部提供）
記者余弦妙／台北報導
因應新冠肺炎（COVID-19）疫情對產業、就業市場及勞工權益造成的影響與衝擊，勞動部啟動各項措施，協助勞工與企業共度難關；由於紓困補助方案較多，勞動部勞動力發展署於官網建置「勞動部因應疫情協助專區」讓勞工更便利查詢。
勞動部勞動力發展署開設懶人專區中，羅列「減班休息勞工」、「失業勞工」、「企業」和「微型創業者」等4大類型常見QA。專區內用淺顯易懂的圖卡說明提供的協助措施、申請流程、申請條件與方式、Q&A等，內容包含「充電再出發訓練計畫」、「失業給付」、「擴大補助企業推動工作與生活平衡措施」等各項方案或措施，且將隨時滾動增加及調整。
發展署表示，本次紓困振興措施相關計畫多已經準備就緒；另外界關心將於3月底實施的「安心就業計畫」及「擴大創業鳳凰貸款對象及提供還款緩衝之優惠措施」相關申請條件與方式也將在專區內提供相關資訊。
勞動部也將持續因應疫情變化制定新協助措施，並隨時滾動調整，提供適切的協助措施，陪伴勞工與企業度過這波衝擊，民眾可至「勞動部勞動力發展署」網站及所屬五分署、技檢中心等網站查詢相關資訊或撥打台灣就業通免付費客服專線0800-777-888洽詢。
----------------------------------------------------------------------

[91] 台股漲幅居亞股之冠！收盤上漲552點　收復9200點

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股今（20）日在三大利多政策以及美股收高的激勵下強勢表態，開高後一路向上攀升。終場收在9234點、上漲552點或6.37%，為亞股中表現最強，成交量2340.27億元。
指標股中，台積電（2330）收盤上漲8.87%、為270元；鴻海（2317）收在70.8元、上漲6.79%；股王大立光（3008）收漲停價3575元；聯發科（2454）也漲停收301元。
國泰證期顧問處分析師簡伯儀協理指出，在三大利多、美股止穩的激勵下，台股出現明顯反彈，並把昨天跌點全數補回。他認為，台股跌破10年線，吸引買盤在此進場承接、搶短，「指數畢竟已跌了3成，出現買盤回升的狀況也合理」。
對於國安基金是否在今日出現進場護盤的跡象，簡伯儀則認為，今天是一開始、應該還不會，但國安基金信心喊話的意圖明顯，